## Using Dynamic Partition Mode

Let us understand how we can insert data into partitioned table using dynamic partition mode.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [1]:
val username = System.getProperty("user.name")

username = itv002461


itv002461

In [2]:
import org.apache.spark.sql.SparkSession

val username = System.getProperty("user.name")
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", s"/user/${username}/warehouse").
    enableHiveSupport.
    appName(s"${username} | Spark SQL - Managing Tables - DML and Partitioning").
    master("yarn").
    getOrCreate

username = itv002461
spark = org.apache.spark.sql.SparkSession@1892f805


org.apache.spark.sql.SparkSession@1892f805

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

* Using dynamic partition mode we need not pre create the partitions. Partitions will be automatically created when we issue INSERT command in dynamic partition mode.
* To insert data using dynamic partition mode, we need to set the property `hive.exec.dynamic.partition` to **true**
* Also we need to set `hive.exec.dynamic.partition.mode` to **nonstrict**

Here is the example of inserting data into partitions using dynamic partition mode.

In [3]:
%%sql

USE itv002461_retail

Waiting for a Spark session to start...

++
||
++
++



In [4]:
%%sql

SHOW tables

+----------------+-----------------+-----------+
|        database|        tableName|isTemporary|
+----------------+-----------------+-----------+
|itv002461_retail|       categories|      false|
|itv002461_retail|        customers|      false|
|itv002461_retail|      departments|      false|
|itv002461_retail|      order_items|      false|
|itv002461_retail|order_items_stage|      false|
|itv002461_retail|           orders|      false|
|itv002461_retail|      orders_part|      false|
|itv002461_retail|         products|      false|
+----------------+-----------------+-----------+



In [5]:
%%sql

SELECT count(1) FROM orders

+--------+
|count(1)|
+--------+
|   68883|
+--------+



In [6]:
%%sql

SELECT count(1) FROM orders_part

+--------+
|count(1)|
+--------+
|   24770|
+--------+



In [7]:
%%sql

SET hive.exec.dynamic.partition

+--------------------+-----------+
|                 key|      value|
+--------------------+-----------+
|hive.exec.dynamic...|<undefined>|
+--------------------+-----------+



In [8]:
%%sql

SET hive.exec.dynamic.partition.mode

+--------------------+-----------+
|                 key|      value|
+--------------------+-----------+
|hive.exec.dynamic...|<undefined>|
+--------------------+-----------+



In [9]:
%%sql

SET hive.exec.dynamic.partition=true

+--------------------+-----+
|                 key|value|
+--------------------+-----+
|hive.exec.dynamic...| true|
+--------------------+-----+



In [10]:
%%sql

SET hive.exec.dynamic.partition.mode=nonstrict

+--------------------+---------+
|                 key|    value|
+--------------------+---------+
|hive.exec.dynamic...|nonstrict|
+--------------------+---------+



In [11]:
%%sql

INSERT INTO TABLE orders_part PARTITION (order_month)
SELECT o.*, date_format(order_date, 'yyyyMM') order_month
FROM orders o
WHERE order_date >= '2013-12-01 00:00:00.0'

++
||
++
++



In [12]:
import sys.process._

s"hdfs dfs -ls -R /user/${username}/warehouse/${username}_retail.db/orders_part" !


-rwxr-xr-x   3 itv002461 supergroup      64737 2022-05-30 01:13 /user/itv002461/warehouse/itv002461_retail.db/orders_part/order_month=201307/orders_201307
drwxr-xr-x   - itv002461 supergroup          0 2022-05-30 01:20 /user/itv002461/warehouse/itv002461_retail.db/orders_part/order_month=201308
-rwxr-xr-x   3 itv002461 supergroup     243190 2022-05-30 01:20 /user/itv002461/warehouse/itv002461_retail.db/orders_part/order_month=201308/orders_201308
drwxr-xr-x   - itv002461 supergroup          0 2022-05-30 01:20 /user/itv002461/warehouse/itv002461_retail.db/orders_part/order_month=201309
-rwxr-xr-x   3 itv002461 supergroup     251262 2022-05-30 01:20 /user/itv002461/warehouse/itv002461_retail.db/orders_part/order_month=201309/orders_201309
drwxr-xr-x   - itv002461 supergroup          0 2022-05-30 01:20 /user/itv002461/warehouse/itv002461_retail.db/orders_part/order_month=201310
-rwxr-xr-x   3 itv002461 supergroup     233173 2022-05-30 01:20 /user/itv002461/warehouse/itv002461_retail.db/o

0

In [13]:
%%sql

SELECT count(1) FROM orders

+--------+
|count(1)|
+--------+
|   68883|
+--------+



In [14]:
%%sql

SELECT count(1) FROM orders_part

+--------+
|count(1)|
+--------+
|   68883|
+--------+



* You will see new partitions created starting from 201312 to 201407.